# Creating a basic chat experience with kernel arguments

In this example, we show how you can build a simple chat bot by sending and updating arguments with your requests. 

We introduce the Kernel Arguments object which in this demo functions similarly as a key-value store that you can use when running the kernel.  

In this chat scenario, as the user talks back and forth with the bot, the arguments get populated with the history of the conversation. During each new run of the kernel, the arguments will be provided to the AI with content. 

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.19.0"
#r "nuget: Azure.Identity, 1.4.0"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Azure.Identity;

var builder = Kernel.CreateBuilder();
DefaultAzureCredentialOptions defaultAzureCredentialOptions = new DefaultAzureCredentialOptions { ExcludeSharedTokenCacheCredential = true, ExcludeEnvironmentCredential = true, ExcludeAzurePowerShellCredential = true, ExcludeInteractiveBrowserCredential = true, ExcludeVisualStudioCredential = true, ExcludeManagedIdentityCredential = true, ExcludeVisualStudioCodeCredential = true, ExcludeAzureCliCredential = false };

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, new DefaultAzureCredential(defaultAzureCredentialOptions));
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Azure.Identity, 1.4.0 Microsoft.SemanticKernel, 1.19.0

Let's define a prompt outlining a dialogue chat bot.

In [2]:
const string skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5
};

Register your semantic function

In [3]:
var chatFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings);

Initialize your arguments

In [4]:
var history = "";
var arguments = new KernelArguments()
{
    ["history"] = history
};

Chat with the Bot

In [5]:
var userInput = "Hi, I'm looking for book suggestions";
arguments["userInput"] = userInput;

var bot_answer = await chatFunction.InvokeAsync(kernel, arguments);

Update the history with the output and set this as the new input value for the next request

In [6]:
history += $"\nUser: {userInput}\nAI: {bot_answer}\n";
arguments["history"] = history;

Console.WriteLine(history);


User: Hi, I'm looking for book suggestions
AI: Hi! I'd be happy to help with book suggestions. Could you tell me a bit about what genres or types of books you enjoy? For example, do you like fiction, non-fiction, mystery, fantasy, science fiction, romance, or something else?



Keep Chatting!

In [7]:
Func<string, Task> Chat = async (string input) => {
    // Save new message in the arguments
    arguments["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(kernel, arguments);

    // Append the new interaction to the chat history
    var result = $"\nUser: {input}\nAI: {answer}\n";
    history += result;

    arguments["history"] = history;
    
    // Show the response
    Console.WriteLine(result);
};

In [8]:
await Chat("I would like a non-fiction book suggestion about Greece history. Please only list one book.");


User: I would like a non-fiction book suggestion about Greece history. Please only list one book.
AI: Certainly! I recommend "The Histories" by Herodotus. It's a classic work that provides a detailed account of ancient Greece and its neighboring regions. Herodotus is often referred to as "The Father of History," and his writings offer a fascinating glimpse into the historical events, cultures, and societies of the time.



In [9]:
await Chat("that sounds interesting, what are some of the topics I will learn about?");


User: that sounds interesting, what are some of the topics I will learn about?
AI: In "The Histories" by Herodotus, you will encounter a wide range of topics related to ancient Greece and its neighboring regions. Some of the key topics include:

1. **The Greco-Persian Wars**: Detailed accounts of the conflicts between the Greek city-states and the Persian Empire, including famous battles like Marathon, Thermopylae, and Salamis.

2. **Cultural Practices**: Descriptions of the customs, traditions, and daily life of various peoples, including the Egyptians, Persians, and Scythians.

3. **Geography and Ethnography**: Insights into the geography of the ancient world and the characteristics of different regions and their inhabitants.

4. **Political History**: Narratives about the rise and fall of empires, the politics of city-states, and the leadership of key figures such as Cyrus the Great, Darius, and Xerxes.

5. **Mythology and Legends**: Accounts of myths, legends, and folklore that we

In [10]:
await Chat("Which topic from the ones you listed do you think most people find interesting?");


User: Which topic from the ones you listed do you think most people find interesting?
AI: While interest can vary greatly from person to person, many readers find the accounts of **The Greco-Persian Wars** particularly fascinating. These sections of "The Histories" are filled with dramatic and pivotal moments, such as the Battle of Marathon, the heroic stand of the Spartans at Thermopylae, and the naval battle of Salamis. These events are not only significant in terms of military history but also in shaping the future of Western civilization. The stories of strategy, bravery, and the clash of cultures often captivate readers and provide a thrilling narrative that stands out in Herodotus's work.



In [11]:
await Chat("could you list some more books I could read about the topic(s) you mentioned?");


User: could you list some more books I could read about the topic(s) you mentioned?
AI: Certainly! Here are some more books that delve into the topics mentioned, particularly focusing on the Greco-Persian Wars and other aspects of ancient Greek history:

1. **"The Persian Wars" by Herodotus** - This is a more focused translation and edition of Herodotus's work, specifically highlighting the Greco-Persian Wars.

2. **"The Landmark Herodotus: The Histories" edited by Robert B. Strassler** - This edition provides extensive maps, annotations, and essays to help readers better understand Herodotus's work.

3. **"The Histories" by Polybius** - Another ancient historian, Polybius provides a detailed account of the rise of the Roman Republic, which includes interactions with Greek states.

4. **"Persian Fire: The First World Empire and the Battle for the West" by Tom Holland** - This book offers a modern and engaging account of the Greco-Persian Wars, providing context and analysis.

5. **"Th

After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!

In [12]:
Console.WriteLine(history);


User: Hi, I'm looking for book suggestions
AI: Hi! I'd be happy to help with book suggestions. Could you tell me a bit about what genres or types of books you enjoy? For example, do you like fiction, non-fiction, mystery, fantasy, science fiction, romance, or something else?

User: I would like a non-fiction book suggestion about Greece history. Please only list one book.
AI: Certainly! I recommend "The Histories" by Herodotus. It's a classic work that provides a detailed account of ancient Greece and its neighboring regions. Herodotus is often referred to as "The Father of History," and his writings offer a fascinating glimpse into the historical events, cultures, and societies of the time.

User: that sounds interesting, what are some of the topics I will learn about?
AI: In "The Histories" by Herodotus, you will encounter a wide range of topics related to ancient Greece and its neighboring regions. Some of the key topics include:

1. **The Greco-Persian Wars**: Detailed accounts of